## Notebook to test basic AI Platform Jupyter Notebook Functionality
This code executes the following steps:
- Obtain data from BigQuery
- Very basic feature engineering for a simplistic way have handling a 
  categorical variable
- Run a very simple random forest regression model model
- Saves the model objects and related files out to Cloud Storage
- Saves output data to Big Query


### Imports and create variables bucket and project name variables

In [1]:

import os
import pickle

import pandas as pd
from IPython.display import display, HTML
from google.cloud import bigquery
from google.cloud import storage

import modeling.data_prep as dp
import modeling.model_train as mt

bucket_path = os.path.expanduser('~/bucket.txt')
with open(bucket_path) as f:
    BUCKET_NAME = f.read().strip()

    project_path = os.path.expanduser('~/project.txt')
with open(project_path) as f:
    PROJECT_NAME = f.read().strip()
    

### Pull a small sample of data from a public data set for testing

In [2]:

client = bigquery.Client()

query = """
    SELECT sale_dollars,
           city,
           county_number,
           category,
           store_number,
           item_number,
           date
      FROM `bigquery-public-data.iowa_liquor_sales.sales`
     LIMIT 1000
"""

query_job = client.query(
    query
)

df = query_job.to_dataframe()

df['year'] = pd.DatetimeIndex(df['date']).year
df['month'] = pd.DatetimeIndex(df['date']).month
df['day'] = pd.DatetimeIndex(df['date']).day

df = df.drop(['date'], axis=1)

print(f'Length of dataframe: {len(df)}')


Length of dataframe: 1000


### Set up the data to run a basic sklearn random forest regression for testing

In [3]:

# Categorical Vars to encode - very simplistic encoding just for testing
cat_vars = [
    'city',
    'category',
    'county_number',
    'store_number',
    'item_number'
]

# Create dataframe with encoded categorical variables
df_cat, df_mapping = dp.category_columns(
    df,
    cat_vars
)

# Round sales_dollar column
df_cat = df_cat.round({'sale_dollars': 2})

print('Sample of Iowa Liquor Sales Data: ')
# noinspection PyTypeChecker
display(HTML(df_cat.head().to_html()))

print('Sample of Mapping Data: ')
display(df_mapping.head())

# Save categorical mapping file - used to make sure mapping is 
# consistently applied 
df_mapping.to_hdf(
    'categorical_mapping.hdf',
    'df_cat_map',
    format='table',
    mode='w'
)

# Save mapping to storage
storage_client = storage.Client()

bucket = storage_client.bucket(BUCKET_NAME)
blob = bucket.blob(
    'ai_platform_test/iowa_forecasting_testing/categorical_mapping.hdf')
blob.upload_from_filename('categorical_mapping.hdf')


Sample of Iowa Liquor Sales Data: 


,sale_dollars,city,county_number,category,store_number,item_number,year,month,day,city_enc,category_enc,county_number_enc,store_number_enc,item_number_enc
0,813.90,ATLANTIC,15,1011500,2591,27102,2012,11,27,0,0,0,0,0
1,50.36,Des Moines,77,1081100,2627,67557,2019,5,23,1,1,1,1,1
2,35.64,RAYMOND,07,1081380,5031,81124,2016,4,18,2,2,2,2,2
3,283.50,DUNLAP,43,1701100,4307,692,2014,7,7,3,3,3,3,3
4,587.16,Dubuque,31,1092100,4167,76227,2018,8,21,4,4,4,4,4


Sample of Mapping Data: 


,city,category,county_number,store_number,item_number,city_enc,category_enc,county_number_enc,store_number_enc,item_number_enc
0,ATLANTIC,1011500,15,2591,27102,0,0,0,0,0
1,Des Moines,1081100,77,2627,67557,1,1,1,1,1
2,RAYMOND,1081380,07,5031,81124,2,2,2,2,2
3,DUNLAP,1701100,43,4307,692,3,3,3,3,3
4,Dubuque,1092100,31,4167,76227,4,4,4,4,4


### Split the data into Train and Test and display samples of the data

In [4]:

# Split data into test and train with 3 months of test data
df_train_rfr, df_test_rfr = mt.split_train_test(
    df_cat,
    'month',
    3
)

print('Length of Training Data: ', len(df_train_rfr))
print('Length of Test Data: ', len(df_test_rfr))

# noinspection PyTypeChecker
display(HTML(df_train_rfr.head().to_html()))
# noinspection PyTypeChecker
display(HTML(df_test_rfr.head().to_html()))

display(df_train_rfr.describe())


Length of Training Data:  559
Length of Test Data:  441


,sale_dollars,city,county_number,category,store_number,item_number,year,month,day,city_enc,category_enc,county_number_enc,store_number_enc,item_number_enc
1,50.36,Des Moines,77,1081100,2627,67557,2019,5,23,1,1,1,1,1
2,35.64,RAYMOND,07,1081380,5031,81124,2016,4,18,2,2,2,2,2
3,283.50,DUNLAP,43,1701100,4307,692,2014,7,7,3,3,3,3,3
4,587.16,Dubuque,31,1092100,4167,76227,2018,8,21,4,4,4,4,4
5,1349.82,West Des Moines,25,1012300,3814,5136,2018,8,16,5,5,5,5,5


,sale_dollars,city,county_number,category,store_number,item_number,year,month,day,city_enc,category_enc,county_number_enc,store_number_enc,item_number_enc
0,813.90,ATLANTIC,15,1011500,2591,27102,2012,11,27,0,0,0,0,0
6,1665.00,Waukee,25,1701100,2665,87026,2019,11,21,6,3,5,6,6
7,404.88,Waukee,25,1042100,2665,28088,2019,11,21,6,6,5,6,7
8,486.00,Fairfield,51,1042100,2601,28866,2019,11,27,7,6,6,7,8
9,46.50,Ankeny,77,1042100,2666,28867,2019,12,3,8,6,1,8,9


,sale_dollars,year,month,day,city_enc,category_enc,county_number_enc,store_number_enc,item_number_enc
count,559.000000,559.000000,559.000000,559.000000,559.000000,559.000000,559.000000,559.000000,559.000000
mean,701.509159,2016.894454,5.425760,16.332737,55.728086,9.908766,18.917710,124.515206,87.837209
std,1394.492019,2.363390,2.189364,9.243821,49.487565,8.872892,18.688649,109.764734,75.970379
min,2.940000,2012.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,51.150000,2015.000000,4.000000,8.000000,18.000000,3.000000,4.000000,30.000000,23.500000
50%,288.000000,2018.000000,6.000000,16.000000,41.000000,6.000000,13.000000,100.000000,57.000000
75%,732.720000,2019.000000,7.000000,25.000000,81.500000,13.000000,29.500000,195.000000,133.000000
max,16284.000000,2020.000000,9.000000,31.000000,203.000000,40.000000,84.000000,408.000000,290.000000


### Run a basic RFR model and save the model object out to GCS

In [5]:

# Set variable we are predicting for and predictors
y_col = 'sale_dollars'
x_cols = [
    'city_enc',
    'county_number_enc',
    'category_enc',
    'store_number_enc',
    'item_number_enc',
    'year',
    'month',
    'day'
]

# Create model object and importances
rfr_model, importances = mt.fit_model(
    y_col,
    x_cols,
    df_train_rfr,
    trees=150,
    leaves=5
)

# Save model object to shared location on edge node
print('Save Model')
pickle.dump(
    rfr_model,
    open('model_test.pkl', 'wb')
)

print('Model saved')

blob = bucket.blob('ai_platform_test/iowa_forecasting_testing/model_test.pkl')
blob.upload_from_filename('model_test.pkl')

print('Model saved to bucket')


Save Model
Model saved
Model saved to bucket


### Using the model and test data, make predictions and save results out to GCS

In [6]:

df_results = mt.model_predict(
    rfr_model,
    df_test_rfr,
    y_col,
    x_cols
)

# Save results file
df_results.to_hdf(
    'model_test_results.hdf',
    'df_results',
    format='table',
    mode='w'
)

# Save mapping to storage
storage_client = storage.Client()

bucket = storage_client.bucket(BUCKET_NAME)
blob = bucket.blob(
    'ai_platform_test/iowa_forecasting_testing/model_test_results.hdf')
blob.upload_from_filename('model_test_results.hdf')


In [7]:
# noinspection PyTypeChecker
display(HTML(df_results.head().to_html()))


,sale_dollars,city,county_number,category,store_number,item_number,year,month,day,city_enc,category_enc,county_number_enc,store_number_enc,item_number_enc,predicted_sale_dollars
0,813.90,ATLANTIC,15,1011500,2591,27102,2012,11,27,0,0,0,0,0,2108.669415
6,1665.00,Waukee,25,1701100,2665,87026,2019,11,21,6,3,5,6,6,2368.525742
7,404.88,Waukee,25,1042100,2665,28088,2019,11,21,6,6,5,6,7,2153.486608
8,486.00,Fairfield,51,1042100,2601,28866,2019,11,27,7,6,6,7,8,2212.405034
9,46.50,Ankeny,77,1042100,2666,28867,2019,12,3,8,6,1,8,9,806.209073


### Also save the results data out to BQ

In [9]:

#Save results to BigQuery
table_id = 'output.ai_platform_ga_test_results'
project_id = PROJECT_NAME
df_results.to_gbq(table_id, project_id, if_exists='replace')


1it [00:02,  2.82s/it]


## ToDo: Remove the BQ ML to it's own notebook
## ToDo: Update code to not be dependent on magic commands

In [ ]:

# Allow you to easily have Python variables in SQL query.
from IPython.core.magic import register_cell_magic
from IPython import get_ipython

@register_cell_magic('with_globals')
def with_globals(line, cell):
    contents = cell.format(**globals())
    if 'print' in line:
        print(contents)
    get_ipython().run_cell(contents)

In [ ]:
%%with_globals
%%bigquery --project $PROJECT_NAME

#Create Model - Dataset being used is the public dataset from above with the encoders 
CREATE or REPLACE MODEL aiplatform_test.liquor_sales_model
OPTIONS
  (model_type='linear_reg',
    input_label_cols=['sale_dollars']) AS
    SELECT sale_dollars,
     city_enc,
     county_number_enc,
     category_enc,
     store_number_enc,
     item_number_enc,
     year,
     month,
     day 
      FROM aiplatform_test.df_cat

In [ ]:
%%with_globals
%%bigquery --project $PROJECT_NAME

#Evaluate Model 
SELECT
    *
FROM
  ML.EVALUATE(MODEL aiplatform_test.liquor_sales_model,
    (
    SELECT
     sale_dollars,
     city_enc,
     county_number_enc,
     category_enc,
     store_number_enc,
     item_number_enc,
     year,
     month,
     day 
      FROM aiplatform_test.df_cat))

In [ ]:
%%with_globals
%%bigquery --project $PROJECT_NAME

#Prediction Outcomes
SELECT
    sale_dollars,
     city, 
     county_number,
     category, 
     store_number, 
     item_number, 
     year, 
     month, 
     day, 
     city_enc,
     county_number_enc,
     category_enc,
     store_number_enc,
     item_number_enc,
     predicted_sale_dollars
FROM
  ML.PREDICT(MODEL aiplatform_test.liquor_sales_model,
    (
    SELECT
     sale_dollars,
     city, 
     county_number,
     category,
     store_number, 
     item_number, 
     year, 
     month, 
     day, 
     city_enc,
     county_number_enc,
     category_enc,
     store_number_enc,
     item_number_enc
      FROM aiplatform_test.df_cat
))

In [ ]:
%%with_globals
%%bigquery --project $PROJECT_NAME

#Create a table with the results 
CREATE TABLE aiplatform_test.bqml_liquor_sales AS (
SELECT
    sale_dollars,
     city, 
     county_number,
     category, 
     store_number, 
     item_number, 
     year, 
     month, 
     day, 
     city_enc,
     county_number_enc,
     category_enc,
     store_number_enc,
     item_number_enc,
     predicted_sale_dollars
FROM
  ML.PREDICT(MODEL aiplatform_test.liquor_sales_model,
    (
    SELECT
     sale_dollars,
     city, 
     county_number,
     category,
     store_number, 
     item_number, 
     year, 
     month, 
     day, 
     city_enc,
     county_number_enc,
     category_enc,
     store_number_enc,
     item_number_enc
      FROM aiplatform_test.df_cat
)))